In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install python-box timm pytorch-lightning==1.4.0 grad-cam ttach

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
# from box import Box
# from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
# from pytorch_grad_cam import GradCAMPlusPlus
# from pytorch_grad_cam.utils.image import show_cam_on_image


# import pytorch_lightning as pl
# from pytorch_lightning.utilities.seed import seed_everything
# from pytorch_lightning import callbacks
# from pytorch_lightning.callbacks.progress import ProgressBarBase
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning import LightningDataModule, LightningModule


warnings.filterwarnings("ignore")

In [ ]:
import seaborn as sns
from pathlib import Path

In [ ]:
df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
# df["Id"] = df["Id"].apply(lambda x: os.path.join(config.root, "train", x + ".jpg"))

In [ ]:
Train_image_dir = '../input/petfinder-pawpularity-score/train/'
Test_image_dir  = '../input/petfinder-pawpularity-score/test/'
Train_f = '../input/petfinder-pawpularity-score/train.csv'
Test_f  = '../input/petfinder-pawpularity-score/test.csv'

In [ ]:
Train_pd = pd.read_csv(Train_f)
Test_pd  = pd.read_csv(Test_f)

In [ ]:
Train_pd.head()

In [ ]:
#show the dimensions of the train metadata.
print('train_f dimensions: ', Train_pd.shape)
print('train_f column names: ', Train_pd.columns.values.tolist())

#print an extra row could use '\n' as well in a print statement
print('')

#show the dimensions of the test metadata
print('test_f dimensions: ',Test_pd.shape)
print('test_f column names: ', Test_pd.columns.values.tolist())

In [ ]:
#Lets see a the distribution of Pawpularity Scores in Train Data
sns.set(rc={'figure.figsize':(15,5)})
fig = plt.figure()
sns.histplot(data=Train_pd, x='Pawpularity', bins=100)
plt.axvline(Train_pd['Pawpularity'].mean(), c='red', ls='-', lw=3, label='Mean Pawpularity')
plt.axvline(Train_pd['Pawpularity'].median(),c='blue',ls='-',lw=3, label='Median Pawpularity')
plt.title('Distribution of Pawpularity Scores', fontsize=20, fontweight='bold')
plt.legend()
plt.show()

In [ ]:
boxplot = Train_pd.boxplot(column='Pawpularity')

In [ ]:
#Calling .describe() lets you see the distribution in a numerical way
Train_pd[['Pawpularity']].describe()

In [ ]:
#Let's start with just one variable to demonstrate
fig, ax = plt.subplots(1,2)
sns.boxplot(data=Train_pd, x='Eyes', y='Pawpularity', ax=ax[0])
sns.histplot(Train_pd, x="Pawpularity", hue="Eyes", kde=True, ax=ax[1])
plt.suptitle("Eyes", fontsize=20, fontweight='bold')
fig.show()

In [ ]:
#Now lets do the same for all the variables with a simple for loop:

#get a the column names into a list
feature_variables = Train_pd.columns.values.tolist()

#for each of the feature variables, doesn't include Id and Pawpularity by using [1:-1]
#show a boxplot and distribution plot against pawpularity
for variable in feature_variables[1:-1]:
    fig, ax = plt.subplots(1,2)
    sns.boxplot(data=Train_pd, x=variable, y='Pawpularity', ax=ax[0])
    sns.histplot(Train_pd, x="Pawpularity", hue=variable, kde=True, ax=ax[1])
    plt.suptitle(variable, fontsize=20, fontweight='bold')
    fig.show()

In [ ]:
print(Train_pd['Pawpularity'].nunique())
print(Train_pd['Pawpularity'].value_counts(sort = True, ascending = True, bins=10))

In [ ]:

from collections import Counter

In [ ]:
unique_image_files = np.unique(Train_pd.Id.values)
print("The number of unique file names : {}".format(len(unique_image_files)))
# print("The distribution of the number of captions for each image:")
# Counter(Counter(Train_pd.Id.values).values())

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

npic = 25
npix = 224
target_size = (npix,npix,3)

count = 1
fig = plt.figure(figsize=(10,20))
# plt.figure(figsize=(12, 12))
for  img in unique_image_files[:npic]:
    filename = Train_image_dir + img + '.jpg'
    captions = list(Train_pd["Pawpularity"].loc[Train_pd["Id"]==img].values)
    image_load = load_img(filename, target_size=target_size)
    plt.subplot(5, 5, count)
    plt.imshow(image_load)
    plt.axis('off')
    plt.title(f'Pawpularity: {captions}')
    count += 1
# plt.show()

In [ ]:
from fastai.vision.all import *

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

In [ ]:
Train_pd['path'] = Train_pd['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')

In [ ]:
Train_pd.head()

In [ ]:
Train_pd['norm_score'] = Train_pd['Pawpularity']/100

In [ ]:
dls = ImageDataLoaders.from_df(Train_pd, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=32, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms

In [ ]:
dls.show_batch()

In [ ]:
# seed=999
# set_seed(seed, reproducible=True)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.backends.cudnn.deterministic = True
# torch.use_deterministic_algorithms = True

In [ ]:
#Sturges' rule
num_bins = int(np.floor(1+np.log2(len(Train_pd))))
num_bins

In [ ]:
Train_pd['bins'] = pd.cut(Train_pd['norm_score'], bins=num_bins, labels=False)
Train_pd['bins'].hist()

In [ ]:
Test_pd = pd.read_csv(Test_f)
Test_pd.head()

In [ ]:
Test_pd['Pawpularity'] = [1]*len(Test_pd)
# Train_pd['path'] = Train_pd['Id'].map(lambda x:str(dataset_path/'train'/x)+'.jpg')
Test_pd['path'] = Test_pd['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
# test_df = test_df.drop(columns=['Id'])
Train_pd['norm_score'] = Train_pd['Pawpularity']/100

In [ ]:
# from timm import create_model

In [ ]:
# Regression Models
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Evalution Metrix
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate
from tqdm import trange
from colorama import Fore

In [ ]:
en = ElasticNet(random_state=0)
gbr = GradientBoostingRegressor(random_state=0)
VR_model = VotingRegressor([('en', en),('gbr', gbr)], n_jobs=-1)

In [ ]:
r2_train = 0; r2_val = 0
rmse_train = 0; rmse_val = 0

In [ ]:
def create_folds_regression(data, target="target", num_splits = 5): 
    """
    Helper function to create folds
    
    """
    data["kfold"] = -1 
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Applying Sturg's rule to calculate the no. of bins for target
    num_bins = int(1 + np.log2(len(data))) 

    data.loc[:, "bins"] = pd.cut(data[target], bins=num_bins, labels=False) 
    
    kf = StratifiedKFold(n_splits=num_splits)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)): 
        data.loc[v_, 'kfold'] = f
        
    data = data.drop(["bins"], axis = 1)         
    return data 


data_df = create_folds_regression(Train_pd, target = 'Pawpularity', num_splits = 5)
data_df.kfold.value_counts()

In [ ]:
req_cols = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]


In [ ]:
def rmse_score(y_label, y_preds):
    """
    Gives RMSE score
    """
    return np.sqrt(mean_squared_error(y_label, y_preds))

In [ ]:
model = VR_model
for fold in trange(5, desc = "Models are training...", bar_format="{l_bar}%s{bar:50}%s{r_bar}" % (Fore.CYAN, Fore.RESET), position = 0, leave = True):
    print(fold)
    Train_pd = data_df.loc[data_df.kfold != fold].reset_index(drop = True)
    val_df = data_df.loc[data_df.kfold == fold].reset_index(drop = True)

    train_X = Train_pd[req_cols]; train_Y = Train_pd["Pawpularity"]
    val_X = val_df[req_cols]; val_Y = val_df["Pawpularity"]
    val_X.head()
    model.fit(train_X, train_Y)

    Y_train_preds = model.predict(train_X)
    Y_val_preds = model.predict(val_X)

    # Collecting the scores
    r2_train += r2_score(train_Y, Y_train_preds)
    r2_val += r2_score(val_Y, Y_val_preds)

    rmse_train += rmse_score(train_Y, Y_train_preds)
    rmse_val += rmse_score(val_Y, Y_val_preds)

print(f"Avg R2 Train Score : {r2_train/5}")
print(f"Avg R2 Val Score : {r2_val/5}")
print(f"Avg RSME Train Score : {rmse_train/5}")
print(f"Avg RSME Val Score : {rmse_val/5}")

In [ ]:
Test_pd

In [ ]:
test_X = Test_pd[req_cols]

model_preds = model.predict(test_X)
Test_pd["Pawpularity"] = model_preds

submission = Test_pd[["Id", "Pawpularity"]]
submission.to_csv("submission.csv", index = False)
data_df.to_csv("data.csv", index = False)
Test_pd.to_csv("test.csv", index = False)
submission